# Problema Principal en el Código 1
### Comportamiento Incorrecto de Autenticación

- Carga del Usuario Incorrecto:
    - Cuando se iniciaba sesión en el Código 1, sin importar con qué cuenta se autenticara el usuario, la aplicación siempre mostraba los datos del desarrollador (o del usuario que estaba originalmente autenticado). Esto sucedía porque:

1. **Uso de cache_path con Valor Fijo:**
    - Se configuraba la instancia de SpotifyOAuth con ``cache_path=".spotify_cache"``.

    - Esto permitía que el token de acceso se almacenara en un archivo en disco.

    - Una vez que el token se obtenía y se guardaba en el caché, cualquier nueva autenticación (incluso si se intentaba usar otra cuenta) reutilizaba ese token cacheado, sobreescribiendo la intención de autenticarse con otra cuenta.

2. **Pérdida y Reinicio del Estado de Sesión:**
    - Se usaba ```st.session_state.clear()``` en varios puntos del código (por ejemplo, después de cargar los datos o en la cuenta regresiva).

    - Esto provocaba que se borrara parte de la información almacenada en ``session_state``, pero el token en caché en disco no se eliminaba.

    - Como resultado, aunque se intentara reiniciar la autenticación, el flujo seguía obteniendo el token ya almacenado (el del desarrollador).

En resumen, el uso del cache_path fijo y el manejo inadecuado de session_state hicieron que, al iniciar sesión, siempre se usara el token ya cacheado (correspondiente al usuario original) y se mostraran sus datos, sin respetar la cuenta con la que el usuario pretendía autenticarse.

### Soluciones Implementadas
1. **Gestión de la Navegación con Pantallas y Control de Estado**

- Variable de Control current_page:
    Se introduce la variable st.session_state["current_page"] para gestionar tres pantallas diferenciadas:

    - "welcome": Pantalla de bienvenida y conexión con Spotify.

    - "first_time": Pantalla para el registro/primer uso, en la que se suben datos a la base de datos.

    - "dashboard": Pantalla principal con el contenido una vez autenticado y registrado.

- Flujo de Navegación Modular:
    - Cada pantalla se implementa en una función separada (por ejemplo, ``pantalla_bienvenida()``, ``pantalla_primera_vez()`` y ``pantalla_dashboard()``), lo que ayuda a mantener el código organizado y a evitar que se mezclen elementos de diferentes etapas.

2. **Manejo Correcto del Token y de la Autenticación**

- Desactivación del Cacheo Persistente:

    - En el Código 2 se configura la instancia de SpotifyOAuth con cache_path=None.

    - Esto evita que se almacene el token en un archivo en disco, obligando a obtener siempre un token actualizado basado en la cuenta que realmente se está utilizando en ese momento.

- Procesamiento y Limpieza de Parámetros de la URL:

    - Se captura el parámetro "code" usando st.query_params y se comprueba si es una lista para tomar el primer valor.

    - Una vez procesado, se limpia la URL con st.query_params.clear(), lo que evita reprocesar el código en futuros reruns.

- Almacenamiento Adecuado del Token e Información del Usuario:
    
    - El token se almacena en ``st.session_state["token_info"]`` y en ``st.session_state["spotify_token"]``.
    
    - La información del usuario (como el ``user_id``) se guarda en ``st.session_state["user_id"]`` para garantizar que se utiliza el usuario actualmente autenticado.

3. **Evitar el Borrado Innecesario del Estado**

    - No se Usa ``st.session_state.clear()``:

    - En el Código 2 se evita el uso de ``st.session_state.clear()``, lo que permite conservar la información crítica (como el token y datos de usuario) a lo largo de la sesión.

    - Esto soluciona el problema en el Código 1 donde, al borrar el estado, se perdían datos y se reutilizaba el token cacheado.

4. **Re-energización de la App con Reruns Controlados**
    
- Uso de ``st.rerun()``:

    - Después de procesar la autenticación y actualizar el estado (por ejemplo, cambiar ``current_page``), se llama a ``st.rerun()`` para reiniciar la aplicación en el nuevo estado.

    - Esto garantiza que la pantalla correspondiente se muestre sin problemas y sin la interferencia de datos antiguos.

# Resumen de los cambios
|Aspecto    |Código Antiguo |Código Corregido
|-----------|------------------|----------------|
Token Cache |	Usa cache_path=".spotify_cache", lo que guarda el token en disco y siempre devuelve el token original (del desarrollador).|Usa ``cache_path=None`` para evitar cacheo persistente y forzar obtener un token actualizado basado en la cuenta del usuario actual.
Manejo del Estado de Sesión |Se utiliza ``st.session_state.clear()`` en varios puntos, lo que puede borrar datos críticos y no afecta el token en caché en disco.|Se evita borrar todo el estado; en su lugar, se utiliza una variable de control (current_page) para gestionar la navegación entre pantallas sin perder información esencial.
Procesamiento de Parámetros|	Se obtiene el parámetro ``"code"`` de la URL pero no se limpia adecuadamente, lo que puede provocar reprocesamientos.| Se captura y limpia el parámetro ``"code"`` con ``st.query_params.clear()``, evitando reprocesos innecesarios.
Flujo de Navegación|	Flujo lineal sin separación clara entre pantallas (bienvenida, primera vez y dashboard).|	Flujo modular con tres funciones separadas para cada pantalla, lo que mejora la claridad y el mantenimiento del código.
Comportamiento de Autenticación|	Siempre se mostraba el usuario original (del desarrollador) debido al token cacheado y manejo inadecuado del estado.|	Se obtiene y almacena correctamente el token e información del usuario, garantizando que se utilice la cuenta que se está autenticando en ese momento.
